# MA_DL : Codierung 64bit

In [1]:
!pip install facenet-pytorch # due to issues with the google cloud service

In [2]:
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models
from PIL import Image

from facenet_pytorch import MTCNN, InceptionResnetV1

dtype = torch.cuda.FloatTensor

## Model

In [3]:
class MultiLabel(nn.Sequential):
    def __init__(self, input_dim=512, output_dim=64):
        super(MultiLabel, self).__init__()
        self.l1 = nn.Linear(input_dim, input_dim)
        self.l2 = nn.Linear(input_dim, input_dim)
        self.l5 = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l5(x)
        return torch.sigmoid(x)

In [4]:
model = MultiLabel()
# model.load_state_dict(torch.load('dl64b_v4_new_414.pt'))
model.cuda()

MultiLabel(
  (l1): Linear(in_features=512, out_features=512, bias=True)
  (l2): Linear(in_features=512, out_features=512, bias=True)
  (l5): Linear(in_features=512, out_features=64, bias=True)
)

## Data

In [20]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()
to_image = transforms.ToPILImage()
transform = transforms.Compose([
    scaler, 
    to_tensor,
])

In [22]:
train_ds = datasets.ImageFolder('../../../data/small_data_new/train/', transform=transform)
train_ds.idx_to_class = {i:c for c, i in train_ds.class_to_idx.items()}
train_dl = DataLoader(train_ds, batch_size=1, num_workers=4)

valid_ds = datasets.ImageFolder('../../../data/small_data_new/valid/', transform=transform)
valid_ds.idx_to_class = {i:c for c, i in valid_ds.class_to_idx.items()}
valid_dl = DataLoader(valid_ds, batch_size=1, num_workers=4)

## Gesichtserkennung

In [7]:
mtcnn = MTCNN(
    image_size=160,
    thresholds=[0.6, 0.7, 0.7],
#        factor=0.709,
#        prewhiten=True,
    keep_all=True,
    device=torch.device('cuda')
    )

## Embeddings 

In [8]:
# Alternativ resnet
face_resnet = InceptionResnetV1(pretrained='casia-webface').eval()

## Training

In [9]:
def calculate_mistakes(pred, target):
    #pred = pred[0]
#    print('calculating mistakes : ')
#    print('   pred: ' + str(pred.shape))
#    print('   target: ' + str(target.shape))
    
    mistakes = 0
    
    if len(pred) != len(target):
        raise Exception('sizes of both tensors must match')
        
    for x,y in zip(pred, target):
        if round(x.item()) != y.item():
            mistakes = mistakes + 1
        
    return mistakes

In [10]:
loss_func = nn.BCELoss() #MultiLabelSoftMarginLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-5, momentum=0.9) #weight_decay=1e-4 

In [11]:
import matplotlib.pyplot as plt

In [23]:
epochs = 1000
for e in range(epochs):
    print("======================")
    print("Epoch : " + str(e))
    epoch_loss = 0
    epoch_mistakes = 0
    train_size = len(train_dl)
    
    # activate train mode
    model.train()

    for index, (data, target) in enumerate(train_dl):
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
                
        target_t = torch.cuda.FloatTensor([int(x) for x in train_ds.idx_to_class[target[0].item()]])

        emb = face_resnet(faces)    
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)
        
        # forward
#        print('datav: ' + str(data_v.shape))
        pred = model.forward(data_v)
#        print('pred: ' + str(pred.shape))
        # zero grads
        optimizer.zero_grad()
        # calculate loss
        loss = loss_func(pred, target_v.float())
        epoch_loss = epoch_loss + loss
        # back prop
        loss.backward()
        optimizer.step()
        
        epoch_mistakes = epoch_mistakes + calculate_mistakes(pred, target_t)
        
    print("Train loss : " + str(epoch_loss.item()))
    print("Average errors : " + str(epoch_mistakes/train_size))
    
    # activate eval mode
    model.eval()
    
    valid_loss = 0
    valid_mistakes = 0
    valid_size = len(valid_dl)
    for index, (data, target) in enumerate(valid_dl):
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
        target_t = torch.FloatTensor([int(x) for x in valid_ds.idx_to_class[target[0].item()]])
        
        emb = face_resnet(faces) 
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)

        pred = model.forward(data_v)
        loss = loss_func(pred, target_v.float())
        valid_loss = valid_loss + loss
        valid_mistakes = valid_mistakes + calculate_mistakes(pred, target_t)
        
        
    print("Valid loss : " + str(valid_loss.item()))
    print("Average valid errors : " + str(valid_mistakes/valid_size))

Epoch : 0
tensor([[[[0.0118, 0.0118, 0.0118,  ..., 0.0078, 0.0118, 0.0118],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.1922, 0.2078, 0.2392,  ..., 0.2392, 0.2431, 0.2471],
          [0.2353, 0.2275, 0.2392,  ..., 0.2431, 0.2471, 0.2510],
          [0.2588, 0.2431, 0.2392,  ..., 0.2431, 0.2471, 0.2510]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0039, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0039, 0.0039,  ..., 0.0000, 0.0039, 0.0039],
          ...,
          [0.1765, 0.1804, 0.1882,  ..., 0.1843, 0.1882, 0.1922],
          [0.2078, 0.1843, 0.1843,  ..., 0.1882, 0.1922, 0.1961],
          [0.2157, 0.1843, 0.1804,  ..., 0.1882, 0.1922, 0.1961]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0039, 0.0039,  ..., 0.0078, 0.0078, 0.0078],
          [0.0118, 0.0118, 0.011

tensor([[[[0.4118, 0.4000, 0.3765,  ..., 0.3922, 0.4078, 0.3961],
          [0.4078, 0.3961, 0.3765,  ..., 0.3961, 0.4157, 0.4039],
          [0.4118, 0.4039, 0.3922,  ..., 0.3922, 0.4078, 0.4039],
          ...,
          [0.0588, 0.0863, 0.1569,  ..., 0.0667, 0.0588, 0.0471],
          [0.0588, 0.0980, 0.1412,  ..., 0.0431, 0.0157, 0.0157],
          [0.0431, 0.0588, 0.0588,  ..., 0.0314, 0.0000, 0.0118]],

         [[0.3216, 0.3098, 0.2863,  ..., 0.2392, 0.2549, 0.2431],
          [0.3176, 0.3059, 0.2863,  ..., 0.2431, 0.2627, 0.2549],
          [0.3216, 0.3137, 0.2980,  ..., 0.2353, 0.2549, 0.2549],
          ...,
          [0.2902, 0.3020, 0.3412,  ..., 0.1569, 0.1569, 0.1412],
          [0.2863, 0.3098, 0.3255,  ..., 0.1373, 0.1059, 0.1098],
          [0.2706, 0.2706, 0.2392,  ..., 0.1176, 0.0941, 0.1059]],

         [[0.1882, 0.1843, 0.1725,  ..., 0.0863, 0.1020, 0.0902],
          [0.1843, 0.1804, 0.1725,  ..., 0.0902, 0.1098, 0.1020],
          [0.1843, 0.1843, 0.1804,  ..., 0

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.1412, 0.1373, 0.1176,  ..., 0.5020, 0.5490, 0.5843],
          [0.1490, 0.1686, 0.1255,  ..., 0.5608, 0.5882, 0.5961],
          [0.1333, 0.1294, 0.1020,  ..., 0.6275, 0.6157, 0.6078]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0392, 0.0392, 0.0392,  ..., 0.5961, 0.6353, 0.6627],
          [0.0471, 0.0706, 0.0471,  ..., 0.6510, 0.6745, 0.6706],
          [0.0314, 0.0314, 0.0196,  ..., 0.7176, 0.6980, 0.6824]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

tensor([[[[0.1255, 0.1176, 0.1020,  ..., 0.0000, 0.0078, 0.0078],
          [0.1608, 0.1529, 0.1412,  ..., 0.0000, 0.0078, 0.0078],
          [0.2275, 0.2118, 0.2000,  ..., 0.0000, 0.0078, 0.0078],
          ...,
          [0.0941, 0.0902, 0.0863,  ..., 0.0784, 0.0745, 0.0745],
          [0.0902, 0.0863, 0.0824,  ..., 0.0784, 0.0784, 0.0745],
          [0.0902, 0.0863, 0.0824,  ..., 0.0784, 0.0784, 0.0745]],

         [[0.0627, 0.0588, 0.0510,  ..., 0.0118, 0.0157, 0.0157],
          [0.0549, 0.0549, 0.0510,  ..., 0.0118, 0.0157, 0.0157],
          [0.0431, 0.0431, 0.0392,  ..., 0.0118, 0.0157, 0.0157],
          ...,
          [0.1137, 0.1098, 0.1059,  ..., 0.0980, 0.0941, 0.0941],
          [0.1098, 0.1059, 0.1020,  ..., 0.0980, 0.0980, 0.0941],
          [0.1098, 0.1059, 0.1020,  ..., 0.0980, 0.0980, 0.0941]],

         [[0.0745, 0.0706, 0.0627,  ..., 0.0078, 0.0118, 0.0118],
          [0.0824, 0.0784, 0.0706,  ..., 0.0078, 0.0118, 0.0118],
          [0.0824, 0.0784, 0.0706,  ..., 0

tensor([[[[0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0039, 0.0039],
          [0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0078, 0.0078],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0863, 0.0902, 0.0980,  ..., 0.0980, 0.1529, 0.2431],
          [0.0902, 0.0980, 0.1020,  ..., 0.1529, 0.2196, 0.3216],
          [0.0941, 0.0980, 0.1020,  ..., 0.2039, 0.2706, 0.3569]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0784, 0.0824, 0.0902,  ..., 0.1176, 0.1725, 0.2627],
          [0.0824, 0.0902, 0.0941,  ..., 0.1725, 0.2471, 0.3490],
          [0.0863, 0.0902, 0.0941,  ..., 0.2235, 0.2941, 0.3843]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0039, 0.0039],
          [0.0000, 0.0000, 0.0000,  ..., 0

...

tensor([[[[0.9216, 0.9216, 0.9216,  ..., 0.6157, 0.6196, 0.6118],
          [0.9216, 0.9216, 0.9216,  ..., 0.6235, 0.6118, 0.5961],
          [0.9216, 0.9216, 0.9216,  ..., 0.6314, 0.6078, 0.5882],
          ...,
          [0.8510, 0.8510, 0.8510,  ..., 0.5020, 0.4980, 0.4980],
          [0.8510, 0.8510, 0.8510,  ..., 0.5020, 0.5020, 0.5020],
          [0.8510, 0.8510, 0.8510,  ..., 0.5020, 0.5020, 0.5020]],

         [[0.9529, 0.9529, 0.9529,  ..., 0.6275, 0.6353, 0.6275],
          [0.9529, 0.9529, 0.9529,  ..., 0.6353, 0.6275, 0.6118],
          [0.9529, 0.9529, 0.9529,  ..., 0.6392, 0.6196, 0.6000],
          ...,
          [0.8745, 0.8745, 0.8745,  ..., 0.4980, 0.4941, 0.4941],
          [0.8745, 0.8745, 0.8745,  ..., 0.4980, 0.4980, 0.4980],
          [0.8745, 0.8745, 0.8745,  ..., 0.4980, 0.4980, 0.4980]],

         [[0.9647, 0.9647, 0.9647,  ..., 0.5333, 0.5373, 0.5294],
          [0.9647, 0.9647, 0.9647,  ..., 0.5412, 0.5294, 0.5137],
          [0.9647, 0.9647, 0.9647,  ..., 0

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0039, 0.0118,  ..., 0.7098, 0.7686, 0.7922],
          [0.0000, 0.0000, 0.0118,  ..., 0.5255, 0.5843, 0.5961],
          [0.0000, 0.0000, 0.0118,  ..., 0.4667, 0.4784, 0.4667]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0039, 0.0118,  ..., 0.5922, 0.6510, 0.6745],
          [0.0000, 0.0000, 0.0118,  ..., 0.3922, 0.4471, 0.4510],
          [0.0000, 0.0000, 0.0118,  ..., 0.3333, 0.3255, 0.3020]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

tensor([[[[ 0.6914,  0.6836,  0.6758,  ..., -0.7461, -0.7539, -0.7617],
          [ 0.6602,  0.6523,  0.6523,  ..., -0.7148, -0.7305, -0.7383],
          [ 0.6289,  0.6289,  0.6367,  ..., -0.6680, -0.6914, -0.7070],
          ...,
          [ 0.5898,  0.5820,  0.5742,  ...,  0.6914,  0.8242,  0.8945],
          [ 0.5820,  0.5820,  0.5820,  ...,  0.7383,  0.8477,  0.9180],
          [ 0.6133,  0.6133,  0.6133,  ...,  0.7695,  0.8555,  0.9102]],

         [[ 0.6680,  0.6602,  0.6523,  ..., -0.6211, -0.6289, -0.6367],
          [ 0.6367,  0.6289,  0.6289,  ..., -0.6055, -0.6211, -0.6289],
          [ 0.6133,  0.6133,  0.6211,  ..., -0.5820, -0.6055, -0.6133],
          ...,
          [-0.1836, -0.2148, -0.2695,  ..., -0.6211, -0.5977, -0.5820],
          [-0.3242, -0.3398, -0.3789,  ..., -0.5977, -0.5742, -0.5664],
          [-0.3867, -0.4023, -0.4414,  ..., -0.5742, -0.5664, -0.5586]],

         [[ 0.3477,  0.3398,  0.3320,  ..., -0.8242, -0.8320, -0.8398],
          [ 0.3164,  0.3164,  

tensor([[[[0.1137, 0.1216, 0.1294,  ..., 0.2392, 0.1804, 0.1490],
          [0.0941, 0.0980, 0.1020,  ..., 0.2118, 0.1569, 0.1255],
          [0.0706, 0.0784, 0.0863,  ..., 0.1804, 0.1216, 0.0863],
          ...,
          [0.1255, 0.1255, 0.1255,  ..., 0.1216, 0.1255, 0.1255],
          [0.1373, 0.1333, 0.1333,  ..., 0.1294, 0.1333, 0.1333],
          [0.1412, 0.1373, 0.1333,  ..., 0.1294, 0.1451, 0.1490]],

         [[0.1216, 0.1333, 0.1451,  ..., 0.2510, 0.1961, 0.1725],
          [0.1176, 0.1176, 0.1255,  ..., 0.2353, 0.1882, 0.1647],
          [0.1098, 0.1176, 0.1255,  ..., 0.2196, 0.1725, 0.1412],
          ...,
          [0.1529, 0.1529, 0.1529,  ..., 0.2039, 0.2078, 0.2039],
          [0.1647, 0.1608, 0.1608,  ..., 0.2039, 0.2118, 0.2078],
          [0.1686, 0.1647, 0.1608,  ..., 0.2000, 0.2196, 0.2196]],

         [[0.0667, 0.0667, 0.0745,  ..., 0.1686, 0.1216, 0.0941],
          [0.0549, 0.0471, 0.0510,  ..., 0.1529, 0.1098, 0.0824],
          [0.0353, 0.0392, 0.0431,  ..., 0

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'dl64b_v4_new_800.pt')

## Testing 

In [62]:
def tensor_to_binary(tensor):
    if tensor.dim() != 2:
        raise ValueException('tensor must have two dimensions')

    for i, x in enumerate(tensor):
        for j, y in enumerate(x) :
            if y.item() < 0.5:
                tensor[i][j] = 0
            elif y.item() >= 0.5:
                tensor[i][j] = 1

    return tensor

In [65]:
for i, t in train_ds:
    print(i.shape)
    faces, prob = mtcnn(to_image(i), return_prob=True)
    target_t = torch.FloatTensor([int(x) for x in valid_ds.idx_to_class[target[0].item()]])
        
    emb = face_resnet(faces) 
    data_v   = Variable(emb[0], requires_grad=False).type(dtype)
    target_v = Variable(target_t, requires_grad=False).type(dtype)

    pred = model.forward(data_v)
    print(pred.shape)
    print(tensor_to_binary(pred.unsqueeze(0)))

torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1.

torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 

torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
         1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
         1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         0., 1., 0., 0., 1., 0., 1., 1., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 1., 1., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 0., 1., 1., 1., 1., 1., 1., 

torch.Size([64])
tensor([[0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1.,
         0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
         0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0.,
         0., 1., 0., 1., 1., 0., 0., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
         0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
         0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 1., 

torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
         1., 0., 0., 1., 0., 1., 0., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
         1., 1., 0., 1., 0., 1., 0., 0., 1., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 1., 1., 1., 

...

torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
         0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
         0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1.,
         1., 0., 0., 1., 0., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
         0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
         0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
         1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
         1., 1., 1., 0., 0., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 1., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 1., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
         0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
         0., 0., 0., 1., 0., 1., 1., 1., 

torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 0., 0., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 0., 0., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 

In [47]:
!ls ../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011

Tom_Hanks_0001.jpg  Tom_Hanks_0004.jpg	Tom_Hanks_0007.jpg  Tom_Hanks_0010.jpg
Tom_Hanks_0002.jpg  Tom_Hanks_0005.jpg	Tom_Hanks_0008.jpg
Tom_Hanks_0003.jpg  Tom_Hanks_0006.jpg	Tom_Hanks_0009.jpg


In [ ]:
path = '../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011/'
for i in os.listdir(path):
        Image.open(path + i)
Image.open('../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011/T')

In [ ]:
model.forward()